In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/FieldNN


# Batch

In [2]:
import os
from fieldnn.dataset import RFGDataset, my_collate_fn
from torch.utils.data import DataLoader

Tensor_folder = 'data/ProcData/FldGrnTensor/'
recfldgrn_list = ['P@PatEcInfo-InfoGrn',  'P-EC-PNSect@AllText-TknzGrn']
full_recfldgrn_list = ['B-' + i for i in recfldgrn_list]

# from the get_grain_fn to get the Elig_Set.
Elig_Set = ['P4', 'P5', 'P6', 'P7']

dataset = RFGDataset(Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID')
print(len(dataset), '<---- dataset')
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True, collate_fn = my_collate_fn)
print(len(dataloader), '<---- dataset')


for idx, batch in enumerate(dataloader):
    # print(f'\n------ {idx}')
    batch_rfg, batch_y = batch
    for k, v in batch_rfg.items(): print(k, v.shape)
    break
    # for k, v in batch_rfg.items(): print(k, v.shape)
    # print(batch_y.shape)

4 <---- dataset
1 <---- dataset
B-P@PatEcInfo-InfoGrn_wgt torch.Size([4, 43])
B-P@PatEcInfo-InfoGrn_tknidx torch.Size([4, 43])
B-P@PatEcInfo-InfoGrn_fldidx torch.Size([4, 43])
B-P-EC-PNSect@AllText-TknzGrn_wgt torch.Size([4, 23, 14, 221])
B-P-EC-PNSect@AllText-TknzGrn_tknidx torch.Size([4, 23, 14, 221])
B-P-EC-PNSect@AllText-TknzGrn_fldidx torch.Size([4, 23, 14, 221])


In [3]:
# Prepare the Input
RECFLD_TO_TENSOR = {}
for full_recfldgrn in full_recfldgrn_list:
    RECFLD_TO_TENSOR[full_recfldgrn] = {k: v for k, v in batch_rfg.items() if full_recfldgrn in k}
    
[i for i in RECFLD_TO_TENSOR]

['B-P@PatEcInfo-InfoGrn', 'B-P-EC-PNSect@AllText-TknzGrn']

In [4]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit

from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_SubUnitName = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_SubUnitName)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P@PatEcInfo-InfoGrn],B-P@PatEcInfo-Info,3,[expander-CateEmbed],"[{'full_recfldgrn': 'B-P@PatEcInfo-InfoGrn', '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
1,E,[B-P-EC-PNSect@AllText-TknzGrn],B-P-EC-PNSect@AllText-Tknz,5,[expander-LLMEmbed],[{'full_recfldgrn': 'B-P-EC-PNSect@AllText-Tkn...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [5]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P@PatEcInfo-Info torch.Size([4, 43, 128])
B-P-EC-PNSect@AllText-Tknz torch.Size([4, 23, 14, 221, 128])


# Get Input Tensors

In [6]:
for full_recfld, info_dict in RECFLD_TO_EMBEDTESNOR.items():
    print(full_recfld, info_dict['info'].shape)

B-P@PatEcInfo-Info torch.Size([4, 43, 128])
B-P-EC-PNSect@AllText-Tknz torch.Size([4, 23, 14, 221, 128])


In [7]:
max_layer = max([len(i.split('-')) for i in RECFLD_TO_EMBEDTESNOR])
max_layer

5

# Get df DataFlow

In [8]:
# fieldnn.utils.datanamefn.py

def get_curfld_recinfo(curfld):
    '''
        a helper function that get the rec information from the current recfldgrn name.
        the input is: prefix_rec@fld_grn
        we want to check whether @ is here. 
        If there is a @ in the current level, we want to put the rec as the potential merger target. 
    '''
    if '@' in curfld:
        fld_list = curfld.split('@')
        return '@'.join(fld_list[:-1]), fld_list[-1]
    else:
        return None, curfld

In [9]:
# fieldnn.dataflowfn.reprflowfn.py

import numpy as np
import pandas as pd


def get_Repr_dataflow_table(full_recfldgrn_list):
    '''
        input: a list of full_recfldgrn
        output: the dataframe that shows the data flow from the grain-embedding-tensor to final feature vector.
    '''
    max_layer = max([len(i.split('-')) for i in full_recfldgrn_list])
    L = []
    for full_recfldgrn in full_recfldgrn_list:
        d = {}
        recfldgrn = '-'.join(full_recfldgrn.split('-')[-2:])
        current_layer_id = len(full_recfldgrn.split('-'))
        d['recfldgrn'] = recfldgrn
        for layer_id in range(max_layer, 1, -1):
            if layer_id > current_layer_id:
                d[layer_id] = np.nan
            elif layer_id == current_layer_id:
                d[layer_id] = full_recfldgrn
            else:
                d[layer_id] = np.nan
        L.append(d)
    df_dataflow = pd.DataFrame(L).set_index('recfldgrn')
    return df_dataflow


In [10]:
def update_df_Repr_dataflow(df, style = 'Reducer&Merger'):
    
    '''
        This function try to auto-fill the data flow from each field to the end field 'B-P'.
        There are two styles:
            style 'Reducer&Merger': Merging the tensors along the data flow.
            style 'ReducerOnly': Don't merge the tensors until the end of the models. 
    
    '''
    
    
    df_dataflow = df.copy()

    for layer_idx in df_dataflow.columns:
        
        # (1) first deal with the fld in each layer
        s = df_dataflow[layer_idx]
        current_fullrfg_list = s[-s.isna()].to_list()
        fld_with_at = [i for i in current_fullrfg_list if '@' in i.split('-')[-1]]
        fld_with_at_to_keep = []
        fld_with_at_to_check = [i for i in fld_with_at if i not in fld_with_at_to_keep]
        
        while len(fld_with_at_to_check) > 0:
            for index in df_dataflow.index:
                full_recfldgrn = df_dataflow.loc[index, layer_idx]
                if pd.isna(full_recfldgrn): continue
                if full_recfldgrn == 'ToFill': continue
                
                if '@' in full_recfldgrn.split('-')[-1]:
                    
                    prefix = '-'.join(full_recfldgrn.split('-')[:-1])
                    curfld = full_recfldgrn.split('-')[-1] # PNSectSent@Sentence@Tk
                    headfld, tailfld = get_curfld_recinfo(curfld) # head: PNSectSent@Sentence, tail: Tk
                    
                    full_recfldgrn_new = full_recfldgrn.replace('@' + tailfld, '')
                    same_rfg_new = [i for i in current_fullrfg_list if full_recfldgrn_new in i]
                    
                    # print(f'for Layer {layer_idx}')
                    # print(full_recfldgrn, '<--- full_recfldgrn')
                    # print(full_recfldgrn_new, '<--- full_recfldgrn_new')
                    # print(same_rfg_new, '<--- same_rfg_new')
                    
                    if len(same_rfg_new) == 1:
                        # print(full_recfldgrn)
                        df_dataflow.loc[index, layer_idx] = full_recfldgrn_new
                        ############# need to think about this
                        s = df_dataflow[layer_idx]
                        current_fullrfg_list = s[-s.isna()].to_list()
                        #############
                        

                    elif len(same_rfg_new) > 1:
                        
                        if style == 'Reducer&Merger':
                            fld_with_at_to_keep.extend(same_rfg_new)
                            
                            # To update a new Column
                            tailfld_merged = '&'.join([i.replace(full_recfldgrn_new +'@', '') for i in same_rfg_new])
                            new_index = '(Merge)' + headfld + '@' + tailfld_merged#
                            if new_index in df_dataflow.index: continue

                            selected_index_list = [df_dataflow[df_dataflow[layer_idx] == i].index[0] for i in same_rfg_new]
                            # print(selected_index_list)

                            l = list(df_dataflow.index)
                            # print(l, '<--- l original columns')
                            loc = max([l.index(i) for i in selected_index_list])
                            l.insert(loc + 1, new_index)
                            # print(l, '<--- l with new index columns')

                            df_dataflow.loc[new_index, layer_idx] = full_recfldgrn_new
                            df_dataflow = df_dataflow.reindex(l)
                            # print(list(df_dataflow.index), '<--- df_dataflow with new index columns')

                            ############# need to think about this
                            # s = df_dataflow[layer_idx]
                            # current_fullrfg_list = s[-s.isna()].to_list()
                            #############
                            fld_with_at_to_keep.append(full_recfldgrn_new)
                            # print(fld_with_at_to_keep, '<----------------'
                            
                        elif style == 'ReducerOnly':
                            fld_with_at_to_keep.extend(same_rfg_new)
                            
                        else:
                            raise ValueError('Wrong style, only for Reducer&Merger and ReducerOnly')
                            
            
            # (2) update conditions
            s = df_dataflow[layer_idx]
            current_fullrfg_list = s[-s.isna()].to_list()
            fld_with_at = [i for i in current_fullrfg_list if '@' in i.split('-')[-1]]
            fld_with_at_to_check = [i for i in fld_with_at if i not in fld_with_at_to_keep]
            
        if layer_idx == df_dataflow.columns[-1]: continue
        # then update the next layer information
        for index in df_dataflow.index:
            last = df_dataflow.loc[index, layer_idx]
            if pd.isna(last): continue
            if '@' in last.split('-')[-1] and style == 'Reducer&Merger': continue
            full_recfldgrn_next = '-'.join(last.split('-')[:-1]) + '@' + last.split('-')[- 1]
            df_dataflow.loc[index, layer_idx - 1] = full_recfldgrn_next
            
            
    if style == 'ReducerOnly':
        df_dataflow.loc['(Merge)P@All', 2] = 'B-P' 
        
    return df_dataflow

In [11]:
def update_df_Repr_dataflow_completename(df):
    df_dataflow = df.copy()
    L = []
    for recfldgrn, row in df_dataflow.iterrows():
        # print(recfldgrn)
        new_row = {}
        new_row['recfldgrn'] = recfldgrn
        full_recfldgrn = [i for i in row.values if not pd.isna(i)][0]
        curlayer_idx = len(full_recfldgrn.split('-'))
        for layer_idx, full_recfldgrn in row.items():
            if curlayer_idx < layer_idx:
                new_row[layer_idx] = np.nan
            elif curlayer_idx == layer_idx:
                new_row[layer_idx] = full_recfldgrn
            elif curlayer_idx > layer_idx:
                last = new_row[layer_idx+1]
                output_fullname = '-'.join(last.split('-')[:-1]) + '@' + last.split('-')[- 1]
                new_row[layer_idx] = output_fullname
            else:
                raise ValueError('wrong information')
        L.append(new_row)
    df_dataflow_filled = pd.DataFrame(L).set_index('recfldgrn')
    return df_dataflow_filled

In [12]:
full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]
full_recfldgrn_list

['B-P@PatEcInfo-Info', 'B-P-EC-PNSect@AllText-Tknz']

In [13]:
df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list)
df_dataflow

,5,4,3,2
recfldgrn,,,,
P@PatEcInfo-Info,NaN,NaN,B-P@PatEcInfo-Info,NaN
PNSect@AllText-Tknz,B-P-EC-PNSect@AllText-Tknz,NaN,NaN,NaN


In [14]:
df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'ReducerOnly')
df_dataflow_new

,5,4,3,2
recfldgrn,,,,
P@PatEcInfo-Info,NaN,NaN,B-P@PatEcInfo-Info,B-P@PatEcInfo
PNSect@AllText-Tknz,B-P-EC-PNSect@AllText-Tknz,B-P-EC-PNSect,B-P-EC,B-P@EC
(Merge)P@All,NaN,NaN,NaN,B-P


In [15]:
df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'Reducer&Merger')
df_dataflow_new

,5,4,3,2
recfldgrn,,,,
P@PatEcInfo-Info,NaN,NaN,B-P@PatEcInfo-Info,B-P@PatEcInfo
PNSect@AllText-Tknz,B-P-EC-PNSect@AllText-Tknz,B-P-EC-PNSect,B-P-EC,B-P@EC
(Merge)P@PatEcInfo&EC,NaN,NaN,NaN,B-P


# Get df SubUnit

In [16]:
df_dataflow = df_dataflow_new.copy()

In [17]:
layeridx_list = list(df_dataflow.columns)
layeridx_list

[5, 4, 3, 2]

In [18]:
for idx in range(len(layeridx_list) - 1):
    A_layerid = layeridx_list[idx]
    B_layerid = layeridx_list[idx + 1]
    print(A_layerid, B_layerid)

5 4
4 3
3 2


In [19]:
for idx in range(len(layeridx_list) - 1):
    A_layerid = layeridx_list[idx]
    B_layerid = layeridx_list[idx + 1]
    # print(A_layerid, B_layerid)
    
    
    A_tensors = df_dataflow[A_layerid]
    A_tensors = A_tensors[-A_tensors.isna()].to_dict()
    
    B_tensors = df_dataflow[B_layerid]
    B_tensors = B_tensors[-B_tensors.isna()].to_dict()
    
    
    print(f'\nFrom Layer {A_layerid} to {B_layerid}:')
    
    # from A tensor to B tensor, we have the Reducer NNs.
    # also notice that some 
    print('A', A_tensors)

    
    # from B tensor: potential there are some Merger NNs. 
    print('B', B_tensors)


From Layer 5 to 4:
A {'PNSect@AllText-Tknz': 'B-P-EC-PNSect@AllText-Tknz'}
B {'PNSect@AllText-Tknz': 'B-P-EC-PNSect'}

From Layer 4 to 3:
A {'PNSect@AllText-Tknz': 'B-P-EC-PNSect'}
B {'P@PatEcInfo-Info': 'B-P@PatEcInfo-Info', 'PNSect@AllText-Tknz': 'B-P-EC'}

From Layer 3 to 2:
A {'P@PatEcInfo-Info': 'B-P@PatEcInfo-Info', 'PNSect@AllText-Tknz': 'B-P-EC'}
B {'P@PatEcInfo-Info': 'B-P@PatEcInfo', 'PNSect@AllText-Tknz': 'B-P@EC', '(Merge)P@PatEcInfo&EC': 'B-P'}


In [20]:

def get_Repr_SubUnit_List(df_dataflow, 
                          default_R_subunit_name = 'RL', 
                          default_MR_subunit_name = 'MRL', # or 'MLRL'
                         ):
    layeridx_list = list(df_dataflow.columns)
    
    SubUnit_List = []
    for idx in range(len(layeridx_list) - 1):
        A_layerid = layeridx_list[idx]
        B_layerid = layeridx_list[idx + 1]
        # print(A_layerid, B_layerid)

        A_tensors = df_dataflow[A_layerid]
        A_tensors = A_tensors[-A_tensors.isna()].to_dict()

        B_tensors = df_dataflow[B_layerid]
        B_tensors = B_tensors[-B_tensors.isna()].to_dict()

        # print(f'\nFrom Layer {A_layerid} to {B_layerid}:')

        # from A tensor to B tensor, we have the Reducer NNs.
        # also notice that some 

        # print('A', A_tensors)
        for k in A_tensors:
            if k not in B_tensors: continue

            input_name = A_tensors[k]
            output_name = B_tensors[k]

            if pd.isna(output_name) == True: continue # pass it. 

            d = {}
            d['SubUnitName'] = default_R_subunit_name
            d['input_names'] = [input_name]
            d['output_name'] = output_name
            
            d['input_layerid'] = A_layerid
            d['output_layerid'] = B_layerid
        
        
            SubUnit_List.append(d)


        # from B tensor: potential there are some Merger NNs. 
        # print('B', B_tensors)
        # check whether these is a '(Merger') in the key
        merger_tensors = [v for k, v in B_tensors.items() if '(Merge)' in k and '@' not in v]
        # print('B-merger_tensors', merger_tensors)

        for output_tensor in merger_tensors:
            input_tensors = [i for k, i in B_tensors.items() if output_tensor + '@' in i]
            # print(output_tensor, ':', input_tensors)

            d = {}
            d['SubUnitName'] = default_MR_subunit_name
            d['input_names'] = input_tensors
            d['output_name'] = output_tensor
            
            d['input_layerid'] = B_layerid
            d['output_layerid'] = B_layerid
        
            SubUnit_List.append(d)
            
    df_SubUnit = pd.DataFrame(SubUnit_List)
    return df_SubUnit




In [21]:
default_R_subunit_name = 'R'
default_MR_subunit_name = 'MR'

df_SubUnit = get_Repr_SubUnit_List(df_dataflow, default_R_subunit_name, default_MR_subunit_name)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,R,[B-P-EC-PNSect@AllText-Tknz],B-P-EC-PNSect,5,4
1,R,[B-P-EC-PNSect],B-P-EC,4,3
2,R,[B-P@PatEcInfo-Info],B-P@PatEcInfo,3,2
3,R,[B-P-EC],B-P@EC,3,2
4,MR,"[B-P@PatEcInfo, B-P@EC]",B-P,2,2
